In [ ]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
from pprint import pprint

In [ ]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

In [ ]:
for x in ratings.take(1).as_numpy_iterator():
  pprint(x)

In [ ]:
for x in movies.take(1).as_numpy_iterator():
  pprint(x)